In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from boardlaw.main import *
from boardlaw.arena import *
snapshot_arena(-1, worldfunc, agentfunc)

In [ ]:
from pavlov import json
import json as _json
json.path(-1, 'arena').write_text(_json.dumps([]))